In [1]:
import os 
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = 'data/scores'

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [4]:
len(box_scores)

8886

In [5]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith('.html')]

In [7]:
def parse_html(box_score):
    with open(box_score, encoding = 'utf-8') as f:
        html = f.read()
        
        soup = BeautifulSoup(html)
        [s.decompose() for s in soup.select('tr.over_header')]
        [s.decompose() for s in soup.select('tr.thead')]
        return soup

In [10]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {'id':'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = 'team'
    cols[-1] = 'total'
    line_score.columns = cols
    line_score = line_score[['team', 'total']]
    return line_score

In [13]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col = 0)[0]
    df = df.apply(pd.to_numeric, errors = 'coerce')
    return df

In [15]:
def read_season_info(soup):
    nav = soup.select('#bottom_nav_container')[0]
    hrefs = [a['href'] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split('_')[0]
    return season

In [28]:
games = []

In [34]:
base_cols = None
for box_score in box_scores[2856:]:
    soup = parse_html(box_score)
    
    line_score = read_line_score(soup)
    teams = list(line_score['team'])
    summaries = []
    
    for team in teams:
        basic = read_stats(soup, team, 'basic')
        advanced = read_stats(soup, team, 'advanced')
        
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()
        
        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"
        
        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep = 'first'))
            base_cols = [b for b in base_cols if 'bpm' not in b]
        
        summary = summary[base_cols]
        summaries.append(summary)
    summary = pd.concat(summaries, axis = 1).T
    
    game = pd.concat([summary, line_score], axis = 1)
    
    game['home'] = [0,1]
    
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += '_opp'
    
    full_game = pd.concat([game, game_opp], axis = 1)
    full_game['season'] = read_season_info(soup)
    
    full_game['date'] = os.path.basename(box_score)[:8]
    full_game['date'] = pd.to_datetime(full_game['date'], format = '%Y%m%d')
    
    full_game['won'] = full_game['total'] > full_game['total_opp']
    games.append(full_game)
    
    if len(games) %  100 == 0:
        print(f'{len(games)} / {len(box_scores)}')

2900 / 8886
3000 / 8886
3100 / 8886
3200 / 8886
3300 / 8886
3400 / 8886
3500 / 8886
3600 / 8886
3700 / 8886
3800 / 8886
3900 / 8886
4000 / 8886
4100 / 8886
4200 / 8886
4300 / 8886
4400 / 8886
4500 / 8886
4600 / 8886
4700 / 8886
4800 / 8886
4900 / 8886
5000 / 8886
5100 / 8886
5200 / 8886
5300 / 8886
5400 / 8886
5500 / 8886
5600 / 8886
5700 / 8886
5800 / 8886
5900 / 8886
6000 / 8886
6100 / 8886
6200 / 8886
6300 / 8886
6400 / 8886
6500 / 8886
6600 / 8886
6700 / 8886
6800 / 8886
6900 / 8886
7000 / 8886
7100 / 8886
7200 / 8886
7300 / 8886
7400 / 8886
7500 / 8886
7600 / 8886
7700 / 8886
7800 / 8886
7900 / 8886
8000 / 8886
8100 / 8886
8200 / 8886
8300 / 8886
8400 / 8886
8500 / 8886
8600 / 8886
8700 / 8886
8800 / 8886


In [33]:
box_scores[2856]

'data/scores\\201711180ORL.html'

In [36]:
games_df = pd.concat(games, ignore_index = True)

In [37]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,...,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
2,240.0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,...,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
3,240.0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,...,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
4,240.0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,...,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17765,240.0,240.0,34.0,85.0,0.400,15.0,38.0,0.395,14.0,19.0,...,42.9,36.3,133.0,112.0,GSW,107,0,2022,2022-06-10,False
17766,240.0,240.0,31.0,75.0,0.413,11.0,32.0,0.344,21.0,31.0,...,33.3,36.2,222.0,107.0,GSW,104,1,2022,2022-06-13,False
17767,240.0,240.0,41.0,88.0,0.466,9.0,40.0,0.225,13.0,15.0,...,45.0,94.4,300.0,112.0,BOS,94,0,2022,2022-06-13,True
17768,240.0,240.0,38.0,92.0,0.413,19.0,46.0,0.413,8.0,8.0,...,100.0,42.6,141.0,126.0,BOS,90,1,2022,2022-06-16,True


In [38]:
games_df.to_csv('nba_games_2015_2022.csv')